# Using Python for GIS and Remote Sensing Operations

### Common Operation
1) Working with Coordinate Systems - Geocoding<br>
2) Web-based Geo data visulization<br>
3) Working with Vector and Raster

## 1) Working with Coordinate Systems - Geocoding

In Mathematics, a coordinate system is a system of numbers (usually called the x-coordinate and the y-coordinate) used to uniquely determine the position of a points, lines and polygons on a plane (graph/grid paper).
<img src='https://4.bp.blogspot.com/-N8YT7c2Lpl4/WYpiWFNuCXI/AAAAAAAABzc/t5RDhEkHZQ8pX0KnUrWWyix8Zo2hzHdZQCLcBGAs/s1600/Cartesian-coordinate-system.svg.png' />
A coordinate system defines the location of a point on a planar or spherical surface
<br>
<u>A Coordinate system</u> in GIS and Remote Sensing enables every location on Earth to be specified by a set of numbers. There are two types of coordinate systems used to represent Geospatial data namely;- <br>
<b>Geographical Coordinate Systems</b> (when the earth is not flat e.g: World Geodetic System 1984 (WGS84). Units are often in Degrees - Longitude/Latitude) and <b>Projected Coordinate Systems</b> (when the earth is flat e.g: Universal Transverse Mercator (UTM) Zone 31. Units are often in Meters - x/y) (<a href='https://en.wikipedia.org/wiki/Geographic_coordinate_system'>source</a>).

<img src='https://2.bp.blogspot.com/-ntdr8jHaDRg/WYSzeZO7_YI/AAAAAAAAByI/y8eGAyZHiHsa1f-YnLs1kFy_aaD2Ib9pQCLcBGAs/s1600/polarcoords.png' />
<br>
<br>

<u>Geocoding:</u> is the process of converting addresses (like a street address) into geographic coordinates (like latitude and longitude), which you can use to place markers on a map, or position the map. <u>Reverse geocoding</u> is the process of converting geographic coordinates into a human-readable address (<a href='https://developers.google.com/maps/documentation/geocoding/start'>source</a>)

<img src='https://2.bp.blogspot.com/-hu_tOXU3bJs/WYl0_BARV3I/AAAAAAAABzM/terg6h-YC3k1K0ZAUQraFDKXgOL_Zt39ACLcBGAs/s1600/3-elders.jpg' />

### Geocoding Places of Tourist Attractions in Lagos

Geopy module makes it easy for Python developers to locate the coordinates (Longitude/Latitude) of addresses, cities, countries, and landmarks across the globe using third-party geocoders and other data sources such as: <br>~ <b>OpenStreetMap Nominatim:</b> https://.nominatim.openstreetmap.org<br>~ <b>ESRI ArcGIS:</b>  https://developers.arcgis.com/rest/geocode/api-reference/overview-world-geocoding-service.htm<br>~ <b>Google Geocoding API (V3):</b> https://.maps.googleapis.com<br>~ Baidu Maps<br>~ Bing Maps API<br>~ Yahoo! PlaceFinder<br>~ Yandex<br>~ IGN France<br>~ GeoNames<br>~ NaviData<br>~ OpenMapQuest<br>~ What3Words<br>~ OpenCage<br>~ SmartyStreets<br>~ geocoder.us <br>~ GeocodeFarm.

<br>
Note that some of the sources above requires you register with them to have api-key.

Source: https://pypi.python.org/pypi/geopy

#### Import Libraries

In [1]:
import folium
import shapefile # pip install pyshp
import pandas as pd
from pyproj import Proj, transform
from geopy.geocoders import Nominatim, ArcGIS, GoogleV3

In [83]:
# Loads the CSV into pandas dataframe
df = pd.read_csv('Data\\Tourist Attractions in Lagos.csv',  encoding='latin1')

In [4]:
# Loop over the places column names and get there long/lat values (Geocoding)
for name in df['Places']:
    try:
        nom = Nominatim(timeout=10) #### Use GeoPy OpenStreetMap Nominatim API:
#         nom = ArcGIS(timeout=10) #### Use GeoPy ESRI ArcGIS API:
#         nom = GoogleV3(timeout=10) #### Use GeoPy Google Geocoding API (V3) API: 
        n = nom.geocode(name)
        print(name, (n.latitude, n.longitude))
        
#         print(n.address)
    except AttributeError:
        print (name, 'None')

Chamber of Commerce and Industry Conference Center, Lagos None
Bar Beach Towers, Lagos None
Slave Trade House, Lagos None
Conservation Centre, Lagos (6.4428238, 3.5351702)
Whispering Palms Resort, Lagos None
National Theatre, Lagos (47.4716412, 19.0704276897326)
Coconut Beach, Lagos (51.94986285, 7.60407079384229)
The Civic Centre, Lagos (6.43985615, 3.43032342465543)
Third Mainland Bridge, Lagos (6.5061582, 3.4036881)
Badagry Slave Route, Lagos None
Freedom Park, Lagos (6.4488245, 3.39653866354212)
Iga Idungaran (Oba's Palace), Lagos None
National Museum, Lagos (52.239433, 21.0361807911491)
Tinubu Square, Lagos (6.5172639, 3.3666241)
Lekki Conservation center, Lagos (6.4428238, 3.5351702)
Tafawa Balewa Square, Lagos (6.4487977, 3.4015766)
Kalakuta Republic Museum, Lagos None


From the geocoding results of the three (3) APIs above, "Google Geocoding API (V3)" gave a better result. So, we use it for further operation.<br>
~ Fewer missing coordinates<br>
~ 90% coodinates value falls within Lagos (Lat: 6.00 - 7.00 and Long: 2.50 - 4.50)

<img src='https://1.bp.blogspot.com/-lDyOD-AAR8Y/WbWTs3WJhjI/AAAAAAAAB0s/vW2nIDX3SnATwoihPekukuqWclFqMYzNwCLcBGAs/s640/Lagos_Coordinate_Bound.PNG' />

#### Load geocoded result into a dataframe/CSV

In [84]:
lat_long_list1 = [] # an empty list to hold lat/long for GoogleV3
lat_long_list2 = [] # an empty list to hold lat/long for ArcGIS API
lat_long_list3 = [] # an empty list to hold lat/long for OpenStreetMap Nominatim

# Loop over the places column names and get there long/lat values (Geocoding)
for name in df['Places']:
#         =========== GoogleV3 API ============    
    try:
        nom1 = GoogleV3(timeout=10)
        n1 = nom1.geocode(name)
        lat_long_list1.append((n1.latitude, n1.longitude))

    except AttributeError:
        lat_long_list1.append(('None', 'None'))

        
#         =========== ArcGIS API ============     
    try:
        nom2 = ArcGIS(timeout=10)
        n2 = nom2.geocode(name)
        lat_long_list2.append((n2.latitude, n2.longitude))

    except AttributeError:
        lat_long_list2.append(('None', 'None'))

        
#         =========== OpenStreetMap Nominatim ============
    try:
        nom3 = Nominatim(timeout=10)
        n3 = nom3.geocode(name)
        lat_long_list3.append((n3.latitude, n3.longitude))

    except AttributeError:
        lat_long_list3.append(('None', 'None'))

        
# convert the "lat_long_list" to dataframe .....
lat_long_df1 = pd.DataFrame(lat_long_list1, columns=['Google-Latitude', 'Google-Longitude'])
lat_long_df2 = pd.DataFrame(lat_long_list2, columns=['ArcGIS-Latitude', 'ArcGIS-Longitude'])
lat_long_df3 = pd.DataFrame(lat_long_list3, columns=['OpenStreetMap-Latitude', 'OpenStreetMap-Longitude'])


# add 'Latitude' and 'Longitude' columns above to the places dataframe
df['Google-Latitude'], df['Google-Longitude'] = lat_long_df1['Google-Latitude'], lat_long_df1['Google-Longitude']
df['ArcGIS-Latitude'], df['ArcGIS-Longitude'] = lat_long_df2['ArcGIS-Latitude'], lat_long_df2['ArcGIS-Longitude']
df['OpenStreetMap-Latitude'], df['OpenStreetMap-Longitude'] = lat_long_df3['OpenStreetMap-Latitude'], lat_long_df3['OpenStreetMap-Longitude']


# save to CSV
df.to_csv("Tourist_Attractions_in_Lagos_with_LatLong.csv", index=None)

df

,Places,Google-Latitude,Google-Longitude,ArcGIS-Latitude,ArcGIS-Longitude,OpenStreetMap-Latitude,OpenStreetMap-Longitude
0,"Bar Beach Towers, Lagos",6.42368,3.41415,6.423480,3.413810,None,None
1,"Slave Trade House, Lagos",None,None,36.363280,-81.725480,None,None
2,"Conservation Centre, Lagos",6.4361,3.53559,-41.289860,174.775230,6.44282,3.53517
3,"Whispering Palms Resort, Lagos",6.41571,3.04479,6.417360,3.043990,None,None
4,"National Theatre, Lagos",6.47644,3.36935,6.485490,3.364810,47.4716,19.0704
5,"Coconut Beach, Lagos",6.65182,3.25978,26.399130,-81.838070,51.9499,7.60407
6,"The Civic Centre, Lagos",6.43988,3.43065,6.439220,3.431100,6.43986,3.43032
7,"Third Mainland Bridge, Lagos",6.50245,3.40251,6.592550,3.383626,6.50616,3.40369
8,"Badagry Slave Route, Lagos",6.40031,2.87547,6.411950,2.883340,None,None
9,"Freedom Park, Lagos",6.4489,3.39648,6.454747,3.338571,6.44882,3.39654


#### Transform "Latitude/Longitude" to "X/Y"
Assuming we care about calculating linear distances between the places, then we have to transform the "Latitude/Longitude" (in Geographical Coordinate Systems) to "X/Y" (in Projected Coordinate Systems).
<br><br>

We know that "Google Geocoding API (V3) API" uses WGS84 Geographical Coordinate Systems, so we need to convert to "Universal Transverse Mercator (UTM) Zone 31" where Lagos state is located.
<br><br>

You can look up the European Petroleum Survey Group (EPSG) code from any traditional GIS tool or online at: http://www.epsg.org/ or https://epsg.io

In [6]:
# defining the input and out Coordinate Systems
inProj = Proj(init='epsg:4326') # WGS84 -- http://epsg.io/4326
outProj = Proj(init='epsg:32631') # UTM Zone 31 -- http://epsg.io/32631

# convert Latitude and Longitude dataframe columns to list
long_list = list(df["Google-Longitude"])
lat_list = list(df["Google-Latitude"])

xy_list = [] # an empty list to hold x/y

for long, lat in zip(long_list, lat_list):
    try:
        x, y = transform(inProj, outProj, long, lat)
#         print (x, y)
        xy_list.append((x, y))
    except Exception:
#         print ("None")
        xy_list.append(('None', 'None'))
        continue

# convert the "xy_list" to dataframe .....
xy_list_df = pd.DataFrame(xy_list, columns=['X', 'Y'])

# add 'X' and 'Y' columns above to the places dataframe
df['X'], df['Y'] = xy_list_df['X'], xy_list_df['Y']

# save to CSV
df.to_csv("Tourist_Attractions_in_Lagos_with_LatLong+XY.csv", index=None)

df

,Places,Google-Latitude,Google-Longitude,ArcGIS-Latitude,ArcGIS-Longitude,OpenStreetMap-Latitude,OpenStreetMap-Longitude,X,Y
0,Chamber of Commerce and Industry Conference Ce...,6.62535,3.36305,41.328260,19.815140,None,None,540130,732348
1,"Bar Beach Towers, Lagos",6.42368,3.41415,6.423480,3.413810,None,None,545798,710058
2,"Slave Trade House, Lagos",None,None,36.363280,-81.725480,None,None,None,None
3,"Conservation Centre, Lagos",6.4361,3.53559,-41.289860,174.775230,6.44282,3.53517,559225,711443
4,"Whispering Palms Resort, Lagos",6.41571,3.04479,6.417360,3.043990,None,None,504953,709159
5,"National Theatre, Lagos",6.47644,3.36935,6.485490,3.364810,47.4716,19.0704,540839,715887
6,"Coconut Beach, Lagos",6.65182,3.25978,26.399130,-81.838070,51.9499,7.60407,528714,735266
7,"The Civic Centre, Lagos",6.43988,3.43065,6.439220,3.431100,6.43986,3.43032,547621,711850
8,"Third Mainland Bridge, Lagos",6.50245,3.40251,6.592550,3.383626,6.50616,3.40369,544504,718765
9,"Badagry Slave Route, Lagos",6.40031,2.87547,6.411950,2.883340,None,None,486228,707458


## 2) Web-based Geo data visulization

This is also-known-as "Web-based GIS or Mapping". There many libraries for this, here we will use the "Folium" libarary to visualize the tourist attraction places above on a web map.

### Tourist Attractions in Lagos

In [87]:
# remove unwanted rows (rows without coordinates)
df = df[df['Google-Latitude'] != 'None']
# df

# convert Places, Latitude and Longitude dataframe columns to list
places_list = list(df['Places'])
long_list = list(df["Google-Longitude"])
lat_list = list(df["Google-Latitude"])

# create folium map object
lagos_map = folium.Map(location=[6.532,3.434], zoom_start=10) # location=[Lat, Long]

# loop through the lists and create markers on the map object
for long, lat, name in zip(long_list, lat_list, places_list):
    lagos_map.add_child(folium.Marker(location=[lat, long], popup=name))
#     lagos_map.add_child(folium.CircleMarker(location=[lat, long], popup=name, radius=5, color='green', fill_color='green', fill_opacity=.2))


# Display the map inline
lagos_map

# you can save the map to html
# lagos_map.save("Lagos_Tourist_Attraction_Map.html")

### Add Conference and Hotel locations

We want to see the closest Tourist Attractions to the Conference Venue/Hotel.

In [122]:
# remove unwanted rows (rows without coordinates)
df = df[df['Google-Latitude'] != 'None']
# df

# convert Places, Latitude and Longitude dataframe columns to list
places_list = list(df['Places'])
long_list = list(df["Google-Longitude"])
lat_list = list(df["Google-Latitude"])

# create folium map object
lagos_map = folium.Map(location=[6.532,3.434], zoom_start=10) # location=[Lat, Long]


# Conference venue
lagos_map.add_child(folium.CircleMarker(location=[6.625292, 3.363003], popup='Lagos Chamber of Commerce and Industry (LCCI)', radius=20, color='green', fill_color='green', fill_opacity=.2))

# Hotel address
lagos_map.add_child(folium.CircleMarker(location=[6.608479, 3.350344], popup='Presken Hotel (Awolowo way)', radius=20, color='red', fill_color='red', fill_opacity=.2))

# loop through the lists and create markers on the map object
for long, lat, name in zip(long_list, lat_list, places_list):
    lagos_map.add_child(folium.Marker(location=[lat, long], popup=name))
#     lagos_map.add_child(folium.CircleMarker(location=[lat, long], popup=name, radius=5, color='green', fill_color='green', fill_opacity=.2))

# ================
# Add a GeoJSON Polygon layer for LGA to see what LGA our Tourist places are located 
# Use "https://mygeodata.cloud/converter" or any GIS tool to covert your data

# geo_json_file = open('data\\Lagos_LGA.geojson', 'r')
# lagos_map.add_child(folium.GeoJson(data=geo_json_file, style_function=lambda x: {'fillColor':'red'}, name="LGA").add_to(lagos_map))
# ================

lagos_map

# you can save the map to html
# lagos_map.save("Lagos_Tourist_Attraction_Map.html")

## 3) Working with Vector and Raster

Some common Vector and Raster file formats used in GIS and RS are "shapefiles"/"GeoJSON" and "TIFF"/"GeoTIFF" files respectively. <a href="http://gisgeography.com/gis-formats/" >Read more...</a>
<br><br>
These libraries: <b>Shapely, Fiona and PyShp</b> are used for working Vector data while libraries such as: <b>GDAL, OpenCV, Scikit-Image, Georasters, GIPPy and Rasterio</b> are used for working Raster data


### Vector data
Lets read in the Lagos shapefile provided in the data folder.

In [68]:
# A shapefile is made up of atleast .shp, .dbf and .shx

# Open shapefile
Lagos_LGA_shp = open("Data\\shapefile\\Lagos_LGA.shp", 'rb') # geometry data
Lagos_LGA_dbf = open("Data\\shapefile\\Lagos_LGA.dbf", 'rb') # property data

# Load shapefile into reader
Lagos_lga = shapefile.Reader(shp=Lagos_LGA_shp, dbf=Lagos_LGA_dbf)
records = Lagos_lga.shapeRecords()

# Checking the number of records in the shapefile (they should be 20 since there are 20 LGAs in Lagos state)
len(records)

# Checking the type of shapefile (Point, Line or Polygon)?
Lagos_lga.shapes()[0].shapeType # 5 for Polygon - https://github.com/GeospatialPython/pyshp
Lagos_lga.shapeType == shapefile.POLYGON # True

# Check attribute fields names
Lagos_lga.fields

# Check bounding box coordinates
Lagos_lga.bbox

# Check attribute records
Lagos_lga.records()

# You can do more reading and writting of ESRI Shapefiles in pure Python - https://github.com/GeospatialPython/pyshp

[['Agege', 461123.0],
 ['Ajeromi/Ifelodun', 745634.0],
 ['Alimosho', 1277714.0],
 ['Amuwo Odofin', 318576.0],
 ['Apapa', 217661.0],
 ['Badagary', 241437.0],
 ['Epe', 181715.0],
 ['Eti-Osa', 287958.0],
 ['Ibeju/Lekki', 117542.0],
 ['Ifako/Ijaye', 428812.0],
 ['Ikeja', 313333.0],
 ['Ikorodu', 535811.0],
 ['Kosofe', 665998.0],
 ['Lagos Island', 209665.0],
 ['Mainland', 317980.0],
 ['Mushin', 633543.0],
 ['Ojo', 598332.0],
 ['Oshodi/Isolo', 621789.0],
 ['Shomolu', 402992.0],
 ['Surulere', 504408.0]]

### Raster Data

At its peak, this is the point where you will play around with Remote Sensing (Satellite Image Processing) tools.

Image processing applies to both satellite images and other kinds of images. Many of the techniques used in satellite images processing are the same used in other software such as Adobe Illustrator or Corel Photo Paint and in the field of computer vision in general.

Some areas of application include:

* Forestry and other land management
* Mineral and oil exploration
* Pollution monitoring
* Urban land use change analysis
* Archaeology:  detection of ancient sites, land use and trade routes
* Flood, fire and other disaster monitoring
* Climatological and oceanographic analysis, such as sea-ice tracking, ocean currents, and ozone monitoring

#### How to get Remote Sensing Data

Remote sensing means observing something from a distance. Satellites in space observe the Earth from a distance and help scientists study large tracts of land and how that land changes over time.

Many Government organazations have satellite that orbits the earth over time. A notable one is the "Landsat Program" which has Over 40 Years of Image Acquisition (It all began with Landsat-1 in 1972. <a href='https://www.nasa.gov/'>NASA</a> and <a href='https://www.usgs.gov/'>USGS</a> are working on Landsat 9 for an expected launch in 2023).

#### Download Landsat data from USGS Earth Explorer

<img src='https://3.bp.blogspot.com/-rChlcPcyhW0/WblJ8LVl9GI/AAAAAAAAB08/xEoMDBEghUAwsBD_EBjkXzzAJkmq8u3sgCLcBGAs/s1600/USGS_Uni_Lagos.PNG' />

<b>GDAL, OpenCV, Scikit-Image, Georasters, GIPPy and Rasterio</b> are some python modules used for working Raster data

In [103]:
img = open("Data\\L1C_T31NEH_A007841_20161222T100906.tif", 'rb')
img

<_io.BufferedReader name='Data\\L1C_T31NEH_A007841_20161222T100906.tif'>

## That's all folks!
# Thank you